Solution:

---
- Problem: Count Numbers with Exactly Three Divisors
- Idea:
    - A number has exactly 3 divisors if and only if it is the square of a prime number.
    - For example: 4 = 2^2 has divisors {1, 2, 4}, 9 = 3^2 has divisors {1, 3, 9}
    
    - Approach 1 (Brute Force):
        - For each number from 1 to n, count its divisors
        - Check if count equals 3
        - Very slow but straightforward
        
    - Approach 2 (Sieve in Query):
        - Generate all primes up to n using Sieve of Eratosthenes
        - Count primes p where p^2 <= n
        - More efficient than brute force
        
    - Approach 3 (Check Each Prime):
        - For each number i from 2 to sqrt(n), check if i is prime
        - Count all prime numbers whose square <= n
        - Uses isPrime function instead of sieve
        
    - Approach 4 (Precomputed Prime Array):
        - Precompute all primes once using sieve
        - Store in array for reuse
        - For query, count primes where p^2 <= n
        - Best for multiple queries
        
    - Approach 5 (Prefix Sum):
        - Precompute sieve and prefix sum of prime counts
        - For query n, answer is prefix[sqrt(n)] in O(1)
        - Fastest for multiple queries
        
- Time Complexity:
    + Approach 1: O(N * sqrt(N)) per query - checking divisors for each number
    + Approach 2: O(N log log N) per query - sieve generation
    + Approach 3: O(N * sqrt(N)) per query - checking primality for each candidate
    + Approach 4: O(N log log N) preprocessing, O(sqrt(N)) per query
    + Approach 5: O(N log log N) preprocessing, O(1) per query
    
- Space Complexity:
    + Approach 1: O(1)
    + Approach 2: O(N)
    + Approach 3: O(1)
    + Approach 4: O(N) for sieve + O(π(N)) for prime array
    + Approach 5: O(N) for sieve and prefix array
---

In [1]:
#include <iostream>
#include <vector>
#include <cmath>     // for sqrtl
#include <cstring>   // for memset
#include <chrono>    // for timing
using namespace std;

class Solution {
public:
    // ---------- Approach 1: Brute Force ----------
    int bruteForce(int n) {
        int total = 0;
        for (int x = 1; x <= n; x++) {
            int count = 0;
            // Count divisors of x
            for (int i = 1; i * i <= x; i++) {
                if (x % i == 0) {
                    count++;
                    if (i != x / i) count++;
                }
            }
            if (count == 3) total++;
        }
        return total;
    }

    // ---------- Approach 2: Sieve of Eratosthenes inside query ----------
    int sieveInQuery(int n) {
        if (n < 4) return 0; // smallest number with 3 divisors is 4 = 2^2
        
        vector<bool> prime(n + 1, true);
        prime[0] = prime[1] = false;
        
        // Sieve to mark all primes
        for (int p = 2; p * p <= n; p++) {
            if (prime[p]) {
                for (int j = p * p; j <= n; j += p)
                    prime[j] = false;
            }
        }
        
        // Count primes whose square <= n
        int total = 0;
        for (int i = 2; i * i <= n; i++) {
            if (prime[i]) total++;
        }
        return total;
    }

    // ---------- Helper function: check if a number is prime ----------
    bool isPrimeSimple(int n) {
        if (n < 2) return false;
        if (n == 2) return true;
        if (n % 2 == 0) return false;
        for (int i = 3; 1LL * i * i <= n; i += 2) {
            if (n % i == 0) return false;
        }
        return true;
    }

    // ---------- Approach 3: Check each prime manually ----------
    int checkEachPrime(int n) {
        if (n < 4) return 0;
        
        int total = 0;
        for (int i = 2; 1LL * i * i <= n; i++) {
            if (isPrimeSimple(i)) total++;
        }
        return total;
    }

    // ---------- Approach 4: Precompute primes using sieve ----------
    bool primeNum[1000001];
    vector<int> primeArray;

    void sieveOfEratosthenes(int N) {
        memset(primeNum, 1, sizeof(primeNum));
        primeNum[0] = primeNum[1] = 0;
        
        for (int i = 2; i * i <= N; i++) {
            if (primeNum[i]) {
                for (int j = i * i; j <= N; j += i)
                    primeNum[j] = 0;
            }
        }
        
        // Store all primes in array
        for (int i = 2; i <= N; i++) {
            if (primeNum[i]) primeArray.push_back(i);
        }
    }

    int solveWithArray(long long n) {
        if (n < 4) return 0;
        
        int counter = 0, index = 0;
        long long currentNum = 1LL * primeArray[index] * primeArray[index];
        
        while (index < (int)primeArray.size() && currentNum <= n) {
            counter++;
            index++;
            if (index < (int)primeArray.size())
                currentNum = 1LL * primeArray[index] * primeArray[index];
        }
        return counter;
    }

    // ---------- Approach 5: Precompute sieve + prefix sum ----------
    int threeDivisorsSingle(long long n) {
        if (n < 4) return 0;
        
        const int MAX = 1000000;
        int limit = min((long long)MAX, n);
        
        vector<bool> isPrime(limit + 1, true);
        vector<int> prefix(limit + 1, 0);

        isPrime[0] = isPrime[1] = false;
        for (int i = 2; i * i <= limit; i++) {
            if (isPrime[i]) {
                for (int j = i * i; j <= limit; j += i)
                    isPrime[j] = false;
            }
        }

        // Build prefix sum array
        for (int i = 1; i <= limit; i++) {
            prefix[i] = prefix[i - 1] + (isPrime[i] ? 1 : 0);
        }

        long long sqrtN = sqrtl(n);
        return prefix[min(sqrtN, (long long)limit)];
    }
    
    vector<int> threeDivisors(vector<long long> query, int q) {
        const int MAX = 1000000;
        vector<bool> isPrime(MAX + 1, true);
        vector<int> prefix(MAX + 1, 0);

        isPrime[0] = isPrime[1] = false;
        for (int i = 2; i * i <= MAX; i++) {
            if (isPrime[i]) {
                for (int j = i * i; j <= MAX; j += i)
                    isPrime[j] = false;
            }
        }

        // Build prefix sum of prime counts
        for (int i = 1; i <= MAX; i++) {
            prefix[i] = prefix[i - 1] + (isPrime[i] ? 1 : 0);
        }

        vector<int> ans;
        for (int i = 0; i < q; i++) {
            long long n = query[i];
            long long limit = sqrtl(n);
            ans.push_back(prefix[min(limit, (long long)MAX)]);
        }
        return ans;
    }
};

Test Case:

In [2]:
Solution sol;

// Test cases
vector<int> testCases = {10, 100, 500, 1000, 10000};

for (int n : testCases) {
    cout << "=====================================" << endl;
    cout << "Test case n = " << n << endl;
    cout << "=====================================" << endl;
    
    // Approach 1: Brute Force (skip for large n)
    if (n <= 1000) {
        auto start = chrono::high_resolution_clock::now();
        int res1 = sol.bruteForce(n);
        auto end = chrono::high_resolution_clock::now();
        auto duration = chrono::duration_cast<chrono::microseconds>(end - start);
        cout << "Approach 1 (Brute Force):    " << res1 
             << " | Time: " << duration.count() << " μs" << endl;
    } else {
        cout << "Approach 1 (Brute Force):    [Skipped - too slow]" << endl;
    }
    
    // Approach 2: Sieve in Query
    auto start2 = chrono::high_resolution_clock::now();
    int res2 = sol.sieveInQuery(n);
    auto end2 = chrono::high_resolution_clock::now();
    auto duration2 = chrono::duration_cast<chrono::microseconds>(end2 - start2);
    cout << "Approach 2 (Sieve in Query): " << res2 
         << " | Time: " << duration2.count() << " μs" << endl;
    
    // Approach 3: Check Each Prime
    auto start3 = chrono::high_resolution_clock::now();
    int res3 = sol.checkEachPrime(n);
    auto end3 = chrono::high_resolution_clock::now();
    auto duration3 = chrono::duration_cast<chrono::microseconds>(end3 - start3);
    cout << "Approach 3 (Check Prime):    " << res3 
         << " | Time: " << duration3.count() << " μs" << endl;
    
    // Approach 5: Prefix Sum (single query)
    auto start5 = chrono::high_resolution_clock::now();
    int res5 = sol.threeDivisorsSingle(n);
    auto end5 = chrono::high_resolution_clock::now();
    auto duration5 = chrono::duration_cast<chrono::microseconds>(end5 - start5);
    cout << "Approach 5 (Prefix Sum):     " << res5 
         << " | Time: " << duration5.count() << " μs" << endl;
    
    cout << endl;
}

// Approach 4: Precomputed Array (best for multiple queries)
cout << "=====================================" << endl;
cout << "Approach 4: Precomputed Prime Array" << endl;
cout << "=====================================" << endl;
auto startPrecompute = chrono::high_resolution_clock::now();
sol.sieveOfEratosthenes(10000);
auto endPrecompute = chrono::high_resolution_clock::now();
auto durationPrecompute = chrono::duration_cast<chrono::microseconds>(endPrecompute - startPrecompute);
cout << "Preprocessing time: " << durationPrecompute.count() << " μs" << endl;

for (int n : testCases) {
    auto start = chrono::high_resolution_clock::now();
    int res = sol.solveWithArray(n);
    auto end = chrono::high_resolution_clock::now();
    auto duration = chrono::duration_cast<chrono::microseconds>(end - start);
    cout << "n = " << n << " → Result: " << res 
         << " | Query time: " << duration.count() << " μs" << endl;
}
cout << endl;

// Multiple queries with Approach 5
cout << "=====================================" << endl;
cout << "Approach 5: Multiple Queries (Batch)" << endl;
cout << "=====================================" << endl;
vector<long long> queries = {10, 100, 500, 1000, 10000, 100000};
auto start = chrono::high_resolution_clock::now();
vector<int> results = sol.threeDivisors(queries, queries.size());
auto end = chrono::high_resolution_clock::now();
auto duration = chrono::duration_cast<chrono::microseconds>(end - start);

cout << "Total time for " << queries.size() << " queries: " << duration.count() << " μs" << endl;
for (int i = 0; i < (int)queries.size(); i++) {
    cout << "n = " << queries[i] << " → Result: " << results[i] << endl;
}

Test case n = 10
Approach 1 (Brute Force):    2 | Time: 0 μs
Approach 2 (Sieve in Query): 2 | Time: 1 μs
Approach 3 (Check Prime):    2 | Time: 0 μs
Approach 5 (Prefix Sum):     2 | Time: 65 μs

Test case n = 100
Approach 1 (Brute Force):    4 | Time: 3 μs
Approach 2 (Sieve in Query): 4 | Time: 3 μs
Approach 3 (Check Prime):    4 | Time: 0 μs
Approach 5 (Prefix Sum):     4 | Time: 5 μs

Test case n = 500
Approach 1 (Brute Force):    8 | Time: 22 μs
Approach 2 (Sieve in Query): 8 | Time: 13 μs
Approach 3 (Check Prime):    8 | Time: 0 μs
Approach 5 (Prefix Sum):     8 | Time: 23 μs

Test case n = 1000
Approach 1 (Brute Force):    11 | Time: 53 μs
Approach 2 (Sieve in Query): 11 | Time: 28 μs
Approach 3 (Check Prime):    11 | Time: 0 μs
Approach 5 (Prefix Sum):     11 | Time: 49 μs

Test case n = 10000
Approach 1 (Brute Force):    [Skipped - too slow]
Approach 2 (Sieve in Query): 25 | Time: 316 μs
Approach 3 (Check Prime):    25 | Time: 0 μs
Approach 5 (Prefix Sum):     25 | Time: 531 μs
